筛选做出Sports->Displine->Event的结构

In [111]:
import pandas as pd
df1 = pd.read_csv('../CleanData/summerOly_programs.csv')
df2 = pd.read_csv('../Source/summerOly_athletes.csv')


In [112]:
discipline_mapping = {}
for _, row in df1.iterrows():
    sport = row['Sport']
    discipline = row['Discipline']
    if sport not in discipline_mapping:
        discipline_mapping[sport] = {}  # 如果 Sport 还没有记录，则初始化为一个空列表
        discipline_mapping[sport]['NAN'] = []  # 并且添加一个空的 Discipline
    discipline_mapping[sport][discipline]=[]  # 将对应的 Discipline 添加到列表中

In [113]:
def get_correct_sport_and_discipline_and_event(row):
    discipline = row['Sport']
    
    for k,v in discipline_mapping.items():
        if discipline == k:
            sport = k
            discipline = 'NAN'
            return pd.Series([sport, discipline, row['Event']], index=['Sport', 'Discipline', 'Event'])
        elif discipline in v:
            sport = k
            return pd.Series([sport, discipline, row['Event']], index=['Sport', 'Discipline', 'Event'])
    #没找到discipline对应的sport,则作为新的sport
    sport = discipline
    discipline_mapping[sport] = {}
    discipline_mapping[sport]['NAN'] = [row['Event']]
    
    return pd.Series([sport, discipline, row['Event']], index=['Sport', 'Discipline', 'Event'])


In [114]:
df2[['Sport', 'Discipline', 'Event']] = df2.apply(get_correct_sport_and_discipline_and_event, axis=1)

In [115]:
df3 = df2[['Sport', 'Discipline', 'Event']]

In [116]:
df3 = df3.copy()
df3.drop_duplicates(keep='first', inplace=True)
len(df3)


1160

In [123]:
# df4 = df3[df3['Sport'] == df3['Discipline']]
# df4['Discipline'] = 'NAN'
# df3 = df3[df3['Sport'] != df3['Discipline']]
# df3 = pd.concat([df3, df4])  
   
import numpy as np

df3['Discipline'] = np.where(df3['Sport'] == df3['Discipline'], 'NaN', df3['Discipline'])

df3 = df3.sort_values(by=[df3.columns[0], df3.columns[1], df3.columns[2]])

df3.to_csv('../WashedType_Only3Column.csv', index=False)
df3.to_excel('../WashedType_Only3Column.xlsx', index=False)

去重

In [127]:
df4 = df3.drop_duplicates(subset=['Sport', 'Event'], keep='last')
df4.to_excel('../WashedType_Only3ColumnLast.xlsx', index=False)

检验差异项

In [126]:
# 筛选出 df1 中不在 df2 中的行
df_diff1 = df3[~df3.isin(df4)].dropna()

# 筛选出 df2 中不在 df1 中的行
df_diff2 = df4[~df4.isin(df3)].dropna()

# 合并这两个 DataFrame
df_diff = pd.concat([df_diff1, df_diff2])
df_diff.to_excel('../WashedType_Only3Column_diff.xlsx', index=False)
